<a href="https://colab.research.google.com/github/Ronaldo-khagokpam/chess/blob/main/chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 82.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=42832e996a245fe1041ec82231a30a6a6bea8afcded92d9f784afb1f101eddbb
  Stored in directory: /root/.cache/pip/wheels/83/1f/4e/8f4300f7dd554eb8de70ddfed96e94d3d030ace10c5b53d447
Successfully built chess


In [ ]:
import chess
import random

# Piece values and simple evaluation
PIECE_VALUES = {chess.PAWN: 1, chess.KNIGHT: 3, chess.BISHOP: 3, chess.ROOK: 5, chess.QUEEN: 9, chess.KING: 0}

PST = {  # Simplified piece-square tables (midgame, white perspective)
    chess.PAWN: [0, 0, 0, 0, 0, 0, 0, 0, 50, 50, 50, 50, 50, 50, 50, 50, 10, 10, 20, 30, 30, 20, 10, 10],
    # Truncated for brevity; expand to 64 squares per piece from sources like Sunfish [web:9]
}

def evaluate_board(board):
    if board.is_game_over():
        if board.is_checkmate():
            return -99999 if board.turn == chess.WHITE else 99999
        return 0
    score = 0
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            value = PIECE_VALUES[piece.piece_type]
            if piece.color == chess.WHITE:
                score += value  # Add PST[piece.piece_type][square] for better eval
            else:
                score -= value
    return score

def minimax(board, depth, alpha, beta, maximizing):
    if depth == 0 or board.is_game_over():
        return evaluate_board(board)
    if maximizing:
        max_eval = -float('inf')
        for move in board.legal_moves:
            board.push(move)
            eval_score = minimax(board, depth - 1, alpha, beta, False)
            board.pop()
            max_eval = max(max_eval, eval_score)
            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for move in board.legal_moves:
            board.push(move)
            eval_score = minimax(board, depth - 1, alpha, beta, True)
            board.pop()
            min_eval = min(min_eval, eval_score)
            beta = min(beta, eval_score)
            if beta <= alpha:
                break
        return min_eval

def find_best_move(board, depth=3):
    best_move = None
    best_value = -float('inf')
    for move in board.legal_moves:
        board.push(move)
        board_value = minimax(board, depth - 1, -float('inf'), float('inf'), False)
        board.pop()
        if board_value > best_value:
            best_value = board_value
            best_move = move
    return best_move

# Main game loop (bot as black)
board = chess.Board()
print("Enter moves in SAN (e.g., e4). Bot plays black.")

while not board.is_game_over():
    print(board)
    if board.turn == chess.WHITE:
        move_san = input("Your move: ")
        move = board.parse_san(move_san)
        board.push(move)
    else:
        move = find_best_move(board)
        board.push(move)
        print(f"Bot move: {move}")
print("Game over:", board.result())  # Outputs "1-0", "0-1", or "1/2-1/2"[web:18]


Enter moves in SAN (e.g., e4). Bot plays black.
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
Your move: e4
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
Bot move: g8h6
r n b q k b . r
p p p p p p p p
. . . . . . . n
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R


In [ ]:
import chess
import chess.engine

# Path to Stockfish executable (update for your OS)
engine_path = "./stockfish.exe"# or "/usr/local/bin/stockfish"

board = chess.Board()
engine = chess.engine.SimpleEngine.popen_uci(engine_path)

# Optional: Weaken Stockfish for fairness (ELO ~1000-2000)
engine.configure({"Skill Level": 10, "Threads": 2, "Hash": 256})[web:37]

print("Play against Stockfish! Enter moves in SAN (e.g., e4). Type 'quit' to exit.")

while not board.is_game_over():
    print(board)

    if board.turn == chess.WHITE:  # User (white) turn
        move_san = input("Your move: ").strip()
        if move_san.lower() == 'quit':
            break
        try:
            move = board.parse_san(move_san)
            board.push(move)
        except ValueError:
            print("Invalid move. Try again.")
            continue
    else:  # Stockfish (black) turn
        print("Stockfish thinking...")
        result = engine.play(board, chess.engine.Limit(time=2.0))  # 2s per move [web:31]
        board.push(result.move)
        print(f"Stockfish: {result.move}")

engine.quit()
print("Game over:", board.result())  # 1-0 win, 0-1 loss, 1/2-1/2 draw [web:18]


In [ ]:
pygame.init()
screen = pygame.display.set_mode((640, 640))
PIECE_IMAGES = {}  # Load: pygame.image.load('chancellor.png')

def draw_board(screen, board):
    for row in range(8):
        for col in range(8):
            color = (240, 217, 181) if (row + col) % 2 == 0 else (181, 136, 99)
            pygame.draw.rect(screen, color, (col*80, row*80, 80, 80))
            piece = board.piece_at(chess.square(col, 7-row))
            if piece:
                screen.blit(PIECE_IMAGES[(piece.symbol(), piece.color)], (col*80, row*80))
    pygame.display.flip()  # Click-to-move integration [web:23][web:26]


In [ ]:
import chess
import random
import pygame  # For optional images

PIECE_VALUES.update({9: 8.5, 10: 9.25})  # Chancellor=9 (R+N), Archbishop=10 (B+N) [web:25]

def evaluate_board(board):  # Handles custom pieces
    score = 0
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            value = PIECE_VALUES.get(piece.piece_type, 1)  # Fallback for customs
            score += value if piece.color == chess.WHITE else -value
    return score  # Expand with custom PSTs [web:9]

# Rest of minimax/find_best_move unchanged...

# Example: Start with custom FEN (Chancellor at e1 for white)
board = chess.Board("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/1NBQKBNR w KQkq - 0 1")  # Custom setup
# Game loop as before, now supports variant moves [web:22]


In [ ]:
import chess
import pygame

# Custom piece class example (extend for minimax eval)
class Chancellor(chess.Piece):
    def __init__(self, color):
        super().__init__(chess.CHANCELLOR, color)  # Custom type 9+

# Simplified custom board (full impl overrides generate_legal_moves)
class CustomBoard(chess.Board):
    def __init__(self):
        super().__init__(chess.variant.CRAZYHOUSE)  # Start with variant base [web:22]
        # Add custom pieces to pockets or board via set_piece_at()
